In [13]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
import pymupdf
import tempfile
from tqdm import tqdm

In [1]:
import juscraper as jus
tjsp = jus.scraper('tjsp')
dados_cjpg = tjsp.cjpg(pesquisa='dano ambiental', paginas=range(1,15))

dados_cjpg

Processando documentos: 100%|██████████| 14/14 [00:01<00:00,  9.58it/s]


,cd_processo,id_processo,classe,assunto,magistrado,comarca,foro,vara,data_disponibilizacao,decisao
0,G1000MS8L0000,1016934-65.2022.8.26.0577,Ação Civil Pública,Ordem Urbanística,Laís Helena de Carvalho Scamilla Jardim,São José dos Campos,Foro de São José dos Campos,2ª Vara da Fazenda Pública,21/05/2025,"SENTENÇA\n\n\n\nConclusão: Em , 08 de maio de ..."
1,1C0004SQX0000,1503965-30.2021.8.26.0048,Ação Penal - Procedimento Ordinário,Parcelamento do solo urbano,Leonardo Marzola Colombini,Atibaia,Foro de Atibaia,3ª Vara Criminal,21/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1503965-...
2,0Z0001WJ80000,1001388-10.2023.8.26.0035,Procedimento Comum Cível,Água e/ou Esgoto,JULIANA MARIA FINATI,Águas de Lindóia,Foro de Águas de Lindoia,Vara Única,21/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\n\n\n\nNú...
3,ER00018DM0000,1000393-32.2021.8.26.0531,Embargos à Execução Fiscal,Extinção da Execução,OTÁVIO AUGUSTO VAZ LYRA,Santa Adélia,Foro de Santa Adélia,Vara Única,20/05/2025,SENTENÇA\n\n\n\nProcesso nº:\t1000393-32.2021....
4,FQ0008PTI0000,1007818-97.2024.8.26.0566,Procedimento Comum Cível,Indenização por Dano Moral,Milton Coutinho Gordo,São Carlos,Foro de São Carlos,1ª Vara Cível,20/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1007818-...
...,...,...,...,...,...,...,...,...,...,...
135,3A0000I0D0000,1000279-76.2018.8.26.0118,Ação Civil Pública,Flora,LUCAS SEMAAN CAMPOS EZEQUIEL,Cananéia,Foro de Cananéia,Vara Única,09/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1000279-...
136,1U000B4FY0000,1009824-25.2024.8.26.0066,Ação Popular,Violação aos Princípios Administrativos,Ricardo Truite Alves,Barretos,Foro de Barretos,1ª Vara Cível,09/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1009824-...
137,7M0001VMN0000,1500627-44.2024.8.26.0274,Crimes Ambientais,Crimes contra a Flora,Ulisses Pizano Vieira Beltrão,Itápolis,Foro de Itápolis,1ª Vara,09/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1500627-...
138,EM000224Y0000,1002623-67.2018.8.26.0526,Procedimento Comum Cível,Aposentadoria Especial (Art. 57/8),ALVARO AMORIM DOURADO LAVINSKY,Salto,Foro de Salto,3ª Vara,09/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1002623-...


In [2]:
dados_cjpg.shape

(140, 10)

In [11]:
sentenca_lista = dados_cjpg.loc[dados_cjpg['assunto'].str.contains('dano ambiental', case=False), :].iloc[0, 9].split('\n')
texto = "".join(sentenca_lista).strip() 

In [12]:
texto = texto.replace('\t', ' ')
texto

'SENTENÇAProcesso Digital nº: 1002294-33.2023.8.26.0024Classe - Assunto Ação Civil Pública - Dano Ambiental Requerente: Fazenda Pública do Estado de São PauloRequerido: Marinalva de Fatima dos SantosJuiz(a) de Direito: Dr(a). GUILHERME FACCHINI BOCCHI AZEVEDOVistos.Trata-se de Ação Civil Pública Ambiental ajuizada pela FAZENDA PÚBLICA DO ESTADO DE SÃO PAULO em face de MARINALVA DE FÁTIMA DOS SANTOS, objetivando a condenação da requerida à recuperação de área de preservação permanente (APP) degradada. Segundo a inicial, no dia 06 de março de 2004, durante fiscalização ambiental realizada no Bairro Porto Independência, município de Castilho/SP, foi constatada construção, reforma e ampliação em alvenaria no imóvel denominado "Rancho do Hugo", suprimindo vegetação natural em área de 0,0192 hectares, considerada de preservação permanente por estar a menos de 100 metros do reservatório da Usina Hidrelétrica Engenheiro Sergio Motta, o que motivou a lavratura do Auto de Infração Ambiental nº 1

In [14]:
def analisa_sentenca(texto_extraido):
    """
    Função que extrai informações de um texto judicial relacionado a danos ambientais.
    O texto deve ser um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
    """
    
    prompt = f"""
        SYSTEM: Você é meu assistente especialista em análise e extração de elementos de textos judiciais. Você irá realizar extrações especificamente sobre
        danos socioambientais de diversos tipos, pensando em futuramente usar a tabela gerada para fazer uma modelagem preditiva de multas para danos socioambientais.

        INSTRUCTIONS:
        Você receberá um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
        Seu objetivo é analisar esse texto e extrair 15 informações específicas para montar um banco de dados, conforme as instruções abaixo.
        Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão
        de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
        Retorno esperado:

        [0] Número do processo judicial no formato "0000000-00.0000.0.00.0000"
         Se não houver, retorne: NULL
        
        [1] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
         Se não houver, retorne: NULL

        [2] Sigla da Unidade Federativa (UF) (ex: "SP", "MG" etc.)
         Se não houver, retorne: NULL

        [3] Município ou cidade do local afetado. (ex: "São Paulo", "Minas Gerais" etc)
         Se não houver, retorne: NULL

        [4] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
         Se não houver, retorne: NULL

        [5] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
         Se não houver, retorne: NULL

        [6] Tipo de impacto: Categoria do dano (ex.: "Desmatamento de APP", "Derramamento de Petróleo", "Poluição Hídrica").
        Se não houver, retorne: NULL

        [7] Descrição do impacto: Resuma o impacto em até 30 palavras.
        Se não houver, retorne: NULL

        [8] Data do impacto ambiental no formato: DD/MM/AA
         Se não houver, retorne: NULL

        [9] Extensão da área afetada (ex: "15000".)
         Se não houver, retorne: NULL
        Sempre em metros quadrados (m²) ou hectares (ha), então converta se necessário. Não use separadores para milhares

        [10] Unidade de medida da área (ex: "ha" ou "m2")
         Se não houver, retorne: NULL

        [11] Houve compensação não monetária atribuída (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
         Se houver, retorne: "True"
         Se não houver, retorne: "False"

        [12] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", "Custas Judiciais e Acordos" ou "Valoração Econômica".
         Se não houver, retorne: NULL
         Tente não fugir dessas categorias.

        [13] Tipo de multa: (0, 1 ou 2)
        Se for uma única multa aplicada sobre o responsável, coloque 0
        Se for algo como uma multa diária, coloque 1.
        Se forem os dois, como uma multa imediata e um pagamento diário, coloque 2

        [14] Valor completo da multa/ressarcimento/compensação ou qualquer outro termo similar de condenação monetária pelo dano ambiental para o Tipo 0 ou 2: (ex: "123000.00", "999999.99")
        Caso o item 12 seja 0, complete essa coluna com o valor total da multa aplicada. Inclua eventuais danos morais difusos, coletivos e materiais se houver.
        Caso o item 12 seja 1, deixe como NULL
        Caso o item 12 seja 2, complete essa coluna com o valor total da multa imediata aplicada.

        [15] Valor da multa/ressarcimento/compensação ou qualquer outro termo similar de condenação monetária pelo dano ambiental diária para o Tipo 1 ou 2: (ex: "123000.00", "10000.50")
        Caso o item 12 seja 1 ou 2, complete essa coluna com o valor diário a ser pago pelo responsável. Inclua eventuais danos morais difusos, coletivos e materiais se houver.
        Caso o item 12 seja 0, deixe como NULL

        IMPORTANTE: A string será usada para um dataframe posteriormente. Logo:
        NÃO inclua explicações, formatações extras, aspas ou colchetes na resposta.
        NÃO deixe nenhuma casa em branco ou vazia. SEMPRE coloque NULL caso não encontre no texto.
        NÃO inclua o texto original, nem explicações, apenas a string limpa.
        A ordem dos elementos PRECISA SER RESPEITADA!! NENHUM ELEMENTO PODE VIR ANTES DA SUA POSIÇÃO!!

        USER:
        Texto para análise:
        {texto_extraido}
    """
    
    class FormatoResposta2(BaseModel):
        numero_processo: str
        georreferencia: str
        uf: str
        municipio: str
        responsavel: str
        categoria_responsavel: str
        tipo_impacto: str
        descricao_impacto: str
        data_impacto: str
        area_afetada: str
        unidade_area: str
        houve_compensacao: bool
        categoria_compensacao: str
        tipo_multa: int | str
        valor_multa: float | str
        valor_multa_diaria: float | str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta2,
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    time.sleep(2)
    return resposta

In [15]:
analise = analisa_sentenca(texto)
analise

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='{\n"numero_processo": "1002294-33.2023.8.26.0024",\n"georreferencia": "NULL",\n"uf": "SP",\n"municipio": "Castilho",\n"responsavel": "Marinalva de Fatima dos Santos",\n"categoria_responsavel": "Pessoa Física",\n"tipo_impacto": "Desmatamento de APP",\n"descricao_impacto": "Construção, reforma e ampliação em alvenaria, suprimindo vegetação natural em área de preservação permanente.",\n"data_impacto": "06/03/04",\n"area_afetada": "192",\n"unidade_area": "m2",\n"houve_compensacao": true,\n"categoria_compensacao": "Obrigações de Fazer (com custo)",\n"tipo_multa": 2,\n"valor_multa": "NULL",\n"valor_multa_diaria": "500.00"\n}')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, avg_l

In [16]:
json.loads(analise.text)  

{'numero_processo': '1002294-33.2023.8.26.0024',
 'georreferencia': 'NULL',
 'uf': 'SP',
 'municipio': 'Castilho',
 'responsavel': 'Marinalva de Fatima dos Santos',
 'categoria_responsavel': 'Pessoa Física',
 'tipo_impacto': 'Desmatamento de APP',
 'descricao_impacto': 'Construção, reforma e ampliação em alvenaria, suprimindo vegetação natural em área de preservação permanente.',
 'data_impacto': '06/03/04',
 'area_afetada': '192',
 'unidade_area': 'm2',
 'houve_compensacao': True,
 'categoria_compensacao': 'Obrigações de Fazer (com custo)',
 'tipo_multa': 2,
 'valor_multa': 'NULL',
 'valor_multa_diaria': '500.00'}